In [1]:
# Import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
# Begin spark session on port 4040
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/23 02:07:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Create custom schema
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [4]:
# Read data into spark cluster
df = spark.read \
    .schema(schema) \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')

In [5]:
df = df.repartition(24)

In [86]:
df.write.parquet('fhvhv/2021')

In [6]:
df = spark.read.parquet('fhvhv/2021')

In [8]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [15]:
df \
    .select('dispatching_base_num', 'pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.dispatching_base_num == 'B02875') \
    .show(2)

+--------------------+-------------------+-------------------+------------+------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+--------------------+-------------------+-------------------+------------+------------+
|              B02875|2021-06-02 22:28:45|2021-06-02 22:37:28|         163|          79|
|              B02875|2021-06-05 02:15:11|2021-06-05 02:35:25|          37|          61|
+--------------------+-------------------+-------------------+------------+------------+
only showing top 2 rows



In [16]:
from pyspark.sql import functions as F

In [ ]:
F.to_date()

In [68]:
def crazy_stuff(location_id):
    return location_id + 1
    

In [69]:
crazy_stuff(126)

127

In [70]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.IntegerType())

In [74]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('pickup_plus', crazy_stuff_udf(df.PULocationID)) \
    .filter(df.dispatching_base_num == 'B02875') \
    .show(2)

24/02/23 02:45:39 ERROR Executor: Exception in task 0.0 in stage 19.0 (TID 19)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 670, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 507, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 289, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 670, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 507, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 289, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/proled/spark/spark-3.3.2-bin-hadoop3/pyt